In [1]:
import folium # library for maping
import json
import pandas as pd
import os
from branca.colormap import LinearColormap

In [ ]:
def filter_zipcodes(input_file,zipcodes,output_file):
    """
    Load geojson data from the input_file, filter the zipcodes with the provided list
    and save it in a new file (output_file)
    """

    # Opening the json file and loading the json content as a dictionnary
    with open(input_file) as file:
        data = json.load(file)
        
    zipData = []
    geometry = []
    dd = 0
    for ii in range(len(data['features'])):
        current_zipcode = data['features'][ii]['properties']['name']     
        if current_zipcode in zipcodes:
#             print(current_zipcode)
            # we also need to clean the structure of the geojson 
            dico = {"type":"Feature",
                   "geometry": data['features'][ii]['geometry'],
                   "properties": {"name":current_zipcode}}
            zipData.append(dico)

    # Creating a new JSON structure to store the data of the selected zipcodes
    new_json = {'type' : 'FeatureCollection',
               'features': zipData}

    # Save into a new json file
    open(output_file,"w").write(
        json.dumps(new_json, separators=(',',':') )
    )
    print("New file saved: "+output_file)

In [2]:
# Resources
# Boundaries of the zipcodes
phx_geo = "resources/phoenix_zip_codes_geo_from_us_az_simplified_ms.json"
phx_geo_filtered = "resources/phoenix_zip_codes_geo_for_PV.json"
# file containing the values to plot
pv_file = os.path.join("resources","clean_pv_2018.csv")

In [3]:
# Loading the Property values
df = pd.read_csv(pv_file)
# df = df.loc[:, ["ZipCode", "Zhvi"]]
df['ZipCode'] = df['ZipCode'].astype(str)
# df = df['ZipCode',"Zhvi"]
# lets try to sort the values
# df = df.sort_values("Zhvi")
df = df.dropna()
# let's just try to remove the NaN rows

df.head()

,ZipCode,City_x,Date,County,City_y,Zhvi
0,85003,Phoenix,2018-08-31,Maricopa,Phoenix,395100.0
1,85004,Phoenix,2018-08-31,Maricopa,Phoenix,262600.0
4,85008,Phoenix,2018-08-31,Maricopa,Phoenix,206400.0
6,85012,Phoenix,2018-08-31,Maricopa,Phoenix,467900.0
7,85013,Phoenix,2018-08-31,Maricopa,Phoenix,305100.0


In [ ]:
# # Lets do this the ugly way
# # we will split the geojson file between the data and no-data values
# zipcodes_with_value = list(df["ZipCode"])

# # loading the geojson file
# with open(phx_geo) as file:
#     data = json.load(file)

# len(zipcodes_with_value)

# filter_zipcodes(phx_geo,zipcodes_with_value,phx_geo_filtered)


In [4]:
# Creating the map


# Transforming the data into a dictionnary (to be used by folium.GeoJson)
# One Entry by zipcode and the correspondig Zhvi value
map_dict = df.set_index('ZipCode')['Zhvi'].to_dict()

# map_dict


In [10]:
m = folium.Map(location = [33.4484, -112.0740], zoom_start=8)
# Adding the colored zipcodes
#  (BuGn, BuPu, GnBu, OrRd, PuBu, PuBuGn, PuRd, RdPu, YlGn, YlGnBu, YlOrBr, and YlOrRd)
m.choropleth(
    geo_data = phx_geo_filtered,  #link to the file containing the geoJSON boundaries of the zipcodes
    data=df,
    columns=['ZipCode', 'Zhvi'],
    key_on='feature.properties.name', # key in the geoJSON structure that contain the zipcode
    fill_color='OrRd',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Zillow estimated median home value',
#     na.value="transparent",
)
folium.LayerControl().add_to(m)
m.save(outfile = "test.html")
m

In [5]:
color_scale = LinearColormap(['yellow','red'], vmin = min(map_dict.values()), vmax = max(map_dict.values()))

In [6]:
def get_color(feature):
    value = map_dict.get(feature['properties']['name'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale(value)
    
filepath = phx_geo
with open(filepath) as ff:
    bla=json.load(ff)

get_color(bla['features'][3])

bli = lambda feature: {
        'fillColor': get_color(feature),
        'fillOpacity': 0.7,
        'color' : 'black',
        'weight' : 1,
    } 

bli(bla['features'][5])

{'fillColor': '#ffe700', 'fillOpacity': 0.7, 'color': 'black', 'weight': 1}

In [16]:
m = folium.Map(location = [33.4484, -112.0740], zoom_start=8.5)
folium.GeoJson(
    data = phx_geo,
    style_function = lambda feature: {
        'fillColor': get_color(feature),
        'fillOpacity': 0.7,
        'color' : 'black',
        'weight' : 1,
    }    
).add_to(m)
color_scale.caption = 'Zillow estimated median home value'
m.add_child(color_scale)


In [8]:
m

In [ ]:
import csv
import numpy as np
# Loading the interesting zipcodes
fziplist = r"resources/zipcodes.csv" # File containing all the zip codes we want to study
cities = [] # list of cities
zipcodes = [] # list of zipcodes
with open(fziplist) as ff:
    zip_list = csv.reader(ff)
    for row in zip_list:
        zipcodes.append(row[0])
        cities.append(row[1])   
df2 = pd.DataFrame({"ZipCode": zipcodes[:10], # [int(xx) for xx in ],
                  "Zhvi": range(len(zipcodes[:10]))})
df2.head()

# df2.dtypes
# type(zipcodes[0])
# df2[""]

In [ ]:
# import random

# map_data = pd.DataFrame({
#     'A3':['POL', 'CZE', 'SVK', 'HUN', 'AUT'],
#     'value':random.sample(range(10), 5)
# })

map_dict = map_data.set_index('A3')['value'].to_dict()
map_data

In [ ]:
color_scale = LinearColormap(['yellow','red'], vmin = min(map_dict.values()), vmax = max(map_dict.values()))


def get_color(feature):
    value = map_dict.get(feature['properties']['A3'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale(value)

m = folium.Map(
    location = [50, 15], 
    zoom_start = 4
)

folium.GeoJson(
    data = 'https://github.com/simonepri/geo-maps/releases/download/v0.6.0/countries-land-10km.geo.json',
    style_function = lambda feature: {
        'fillColor': get_color(feature),
        'fillOpacity': 0.7,
        'color' : 'black',
        'weight' : 1,
    }    
).add_to(m)

In [ ]:
m

In [ ]:
# Adding the colored zipcodes
m.choropleth(
    geo_data = phx_geo,  #link to the file containing the geoJSON boundaries of the zipcodes
    data=df,
    columns=['ZipCode', 'Zhvi'],
    key_on='feature.properties.name', # key in the geoJSON structure that contain the zipcode
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Zillow estimated median home value',
#     na.value="transparent",
)
folium.LayerControl().add_to(m)
m.save(outfile = "test.html")

In [ ]:
m

In [ ]:
import csv
import numpy as np
# Loading the interesting zipcodes
fziplist = r"resources/zipcodes.csv" # File containing all the zip codes we want to study
cities = [] # list of cities
zipcodes = [] # list of zipcodes
with open(fziplist) as ff:
    zip_list = csv.reader(ff)
    for row in zip_list:
        zipcodes.append(row[0])
        cities.append(row[1])   
df = pd.DataFrame({"Zipcode": zipcodes,
                  "Value": range(len(zipcodes))})

In [ ]:
df.head()

In [ ]:
m

In [ ]:
# Loading zipcodes data
# with open("phoenix_zip_codes_geo.json") as jsonFile:
#     data = json.load(jsonFile)

m.choropleth(
    geo_data = phx_geo,
    data=df,
     columns=['Zipcode', 'Value'],
     key_on='feature.properties.name',
     fill_color='YlGn',
     fill_opacity=0.7,
     line_opacity=0.2,
     legend_name='blabla'

)
folium.LayerControl().add_to(m)
m.save(outfile = "test.png")

In [ ]:
m